<a href="https://colab.research.google.com/github/piesauce/llm-playbooks/blob/main/notebooks/Loading_Models_with_BnB_in_Colab_Free_Tier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Models

## Resource limitations in free tier
The selection of models assumes that we have access to the following resources (which can vary!):
* 78.2GB disk space (!)
* 15.0GB GPU RAM (!)
* 12.7GB system RAM

System RAM is not the most important factor here since we are loading our models in GPU, but the disk space and GPU RAM is crucial. Here is how they scale w.r.t. the number of model parameters:

* Assuming model weights are stored in 16-bit float or [bfloat](https://en.wikipedia.org/wiki/Bfloat16_floating-point_format) format, each 1B parameters will require 2GB of disk space.

* Assuming models are loaded into 4-bit format and no sharding is involved, each 1B parameters will require 500MB of GPU RAM.

Therefore the Llama 2 model size that free tier Colab can support without sharding is 13B (26GB disk space, 6.5GB GPU RAM). This does not yet account for any buffer or overhead that may otherwise also be needed by the system.

Since the 70B model will require 140GB of disk space and 35GB of GPU RAM, we will not be able to load it here.

## Llama 2 permissions and OPT-350M
We also put in OPT-350M as a sanity check since Llama 2 requires requesting permissions from Meta to run ([request here](https://huggingface.co/meta-llama/Llama-2-7b)) and that can take a few hours or more to be granted.

In [4]:
MODEL_IDS = {
    # strictly a sanity check: this model is too small and not instruction tuned
    "opt-350m":         "facebook/opt-350m",

    # need to request permissions to load Llama 2 models
    "llama2-7b":        "meta-llama/Llama-2-7b-hf",
    "llama2-7b-chat":   "meta-llama/Llama-2-7b-chat-hf",
    "llama2-13b":       "meta-llama/Llama-2-13b-hf",
    "llama2-13b-chat":  "meta-llama/Llama-2-13b-chat-hf",
}

In [3]:
# Llama 2 models require permissions to load; be sure to log in
!git config --global credential.helper store

from huggingface_hub import notebook_login
notebook_login()

# Boring setup stuff

* Colab and Jupyter notebooks in general don't format output text very well; we use `textwrap` to manually print out text in a readable format.
* We need to update some HuggingFace libraries and install `bitsandbytes` manually.

In [1]:
# TODO: put this util into the repo
import textwrap

def print_wrap(text: str, width: int = 100, subsequent_indent: str = "ꜛ"):
    """
    Wrap and print text to some max width.

    Useful for notebooks which do not wrap output text, making it very hard to read
    in both Colab and GitHub.
    """
    print(
        "\n".join(
            textwrap.fill(line, subsequent_indent=subsequent_indent, width=width)
            for line in text.split("\n")
        )
    )


text = """In my project, I have a bunch of strings that are.
Read in from a file.
Most of them, when printed in the command console, exceed 80. Characters in length and wrap around, looking ugly."""

print_wrap(text)

In my project, I have a bunch of strings that are.
Read in from a file.
Most of them, when printed in the command console, exceed 80. Characters in length and wrap around,
ꜛlooking ugly.


In [2]:
!pip install -U bitsandbytes --root-user-action=ignore
!pip install -U git+https://github.com/huggingface/transformers.git --root-user-action=ignore
!pip install -U git+https://github.com/huggingface/peft.git --root-user-action=ignore
!pip install -U git+https://github.com/huggingface/accelerate.git --root-user-action=ignore
!pip install -U tokenizers

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-7adaj1hj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-7adaj1hj
  Resolved https://github.com/huggingface/transformers.git to commit d8e13b3e04da9e61c6f16df43815656f59688abd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-ebx9u465
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-ebx9u465
  Resolved https://github.com/huggingface/peft.git to commit 0c9354bda98eb7f5348699e23ab752e8dca1e60e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-zbtt2xb_
  Runni

# Loading and running model

Now we will load our selected model and see its outputs in a question answering setting.

Due to resource constraints, we simply cannot load and run all models in a single runtime instance. So we will need to clear the model from GPU memory every time and clear disk space as needed.

We will also be loading our models in 4-bit in order to minimize usage, although it should also be possible to load the 7B model in 8-bit format using [`BitesAndBytesConfig(load_in_8bit=True, ...)`](https://huggingface.co/docs/transformers/v4.33.0/en/main_classes/quantization#transformers.BitsAndBytesConfig).

In [5]:
# Clear disk space as needed
from transformers import file_utils
from shutil import rmtree

rmtree(file_utils.default_cache_path, ignore_errors=True)

In [6]:
# Config
model_id = MODEL_IDS["llama2-13b-chat"]     # change the model here

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model
from torch import cuda, bfloat16
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import transformers

try:
    # If an existing model is loaded, clear it from GPU memory
    del model
except NameError:
    pass

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Now comes the fun part: let us see what the model outputs for a simple Q&A problem instance.

To do this we need to set up a simple QA prompt. For pedagogical purposes we will split up the prompt into some constituent components
* `instruction_text`: This helps the model frame the task and the constraints around the task. Considering how common `Question: <...>, Answer: <...>` text is, this is probably not strictly necessary.
* `input_text`: Here is where we ask the specific question that we want the answer to. Notice the `Question:` prefix.
* `output_prefix`: By providing the `Answer:` prefix, we help to condition the model so that it will generate tokens to answer the question. Without this it is possible that the model will continue generating text for the question, which is not what we want.

Feel free to change the model and prompt and see how this affects the generated output!

In [10]:
def qa_prompt(question: str) -> str:
    instruction_text = (
        "You are a helpful, respectful and honest assistant. "
        "Always answer as helpfully as possible, while being safe. "
        "Your answers should not include any harmful, unethical, racist, sexist, toxic, "
        "dangerous, or illegal content. Please ensure that your responses are socially unbiased "
        "and positive in nature."
        "\n\n"
        "If a question does not make any sense, or is not factually coherent, explain why instead "
        "of answering something not correct. If you don't know the answer to a question, please "
        "don't share false information."
    )
    input_text = (
        "Question:\n"
        + question
    )
    output_prefix = "Answer:\n"
    return "\n\n".join([instruction_text, input_text, output_prefix])

device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=1000)
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print_wrap(output_text)

You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while
ꜛbeing safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic,
ꜛdangerous, or illegal content. Please ensure that your responses are socially unbiased and positive
ꜛin nature.

If a question does not make any sense, or is not factually coherent, explain why instead of
ꜛanswering something not correct. If you don't know the answer to a question, please don't share
ꜛfalse information.

Question:
How do you load a BERT model for sentiment classification using Hugging Face transfomers?

Answer:
To load a BERT model for sentiment classification using Hugging Face transformers, you can follow
ꜛthese steps:

1. First, make sure you have the Hugging Face transformers library installed. You can install it by
ꜛrunning the following command in your terminal or command prompt:
```
pip install transformers
```
2. Next, you need to download the BERT model. You can do this b